In [165]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [166]:
import torch.nn as nn
import torch
from torch.utils import data
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from scripts.train import *

from sklearn.model_selection import train_test_split



batch_size = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [167]:
all_images = [f for f in os.listdir('./data/images') if f.endswith(('.jpg', '.png', '.jpeg'))]

len(all_images)

7390

In [168]:
df = pd.read_csv('data/annotations/list.txt', sep=" ")
df.class_id = df.class_id - 1

df['image'] = df['image'].map({os.path.splitext(f)[0]: f for f in os.listdir('data/images/')}) #Пишет полное название файла
train, test = train_test_split(df[['image', 'class_id']], stratify= df.class_id)




# df.groupby(by = ['class_id', 'split']).count()

In [169]:
train

,image,class_id
5804,miniature_pinscher_51.jpg,21
6488,saint_bernard_37.jpg,28
5706,Maine_Coon_44.jpg,20
5199,great_pyrenees_46.jpg,15
1392,Russian_Blue_151.jpg,27
...,...,...
3598,wheaten_terrier_165.jpg,35
6155,pomeranian_98.jpg,24
3749,Abyssinian_76.jpg,0
2527,english_setter_187.jpg,13


In [170]:


train_files = (train.image)
test_files = (test.image)

train_indicies = (train.index)
test_indicies = (test.index)

In [171]:
train

,image,class_id
5804,miniature_pinscher_51.jpg,21
6488,saint_bernard_37.jpg,28
5706,Maine_Coon_44.jpg,20
5199,great_pyrenees_46.jpg,15
1392,Russian_Blue_151.jpg,27
...,...,...
3598,wheaten_terrier_165.jpg,35
6155,pomeranian_98.jpg,24
3749,Abyssinian_76.jpg,0
2527,english_setter_187.jpg,13


In [172]:
for file, label in zip(df.image, df.class_id):
    print(file, label)

Abyssinian_103.jpg 0
Abyssinian_104.jpg 0
Abyssinian_105.jpg 0
Abyssinian_106.jpg 0
Abyssinian_107.jpg 0
Abyssinian_108.jpg 0
Abyssinian_109.jpg 0
Abyssinian_10.jpg 0
Abyssinian_110.jpg 0
Abyssinian_111.jpg 0
Abyssinian_112.jpg 0
Abyssinian_113.jpg 0
Abyssinian_114.jpg 0
Abyssinian_115.jpg 0
Abyssinian_116.jpg 0
Abyssinian_117.jpg 0
Abyssinian_118.jpg 0
Abyssinian_119.jpg 0
Abyssinian_11.jpg 0
Abyssinian_120.jpg 0
Abyssinian_121.jpg 0
Abyssinian_122.jpg 0
Abyssinian_123.jpg 0
Abyssinian_124.jpg 0
Abyssinian_125.jpg 0
Abyssinian_126.jpg 0
Abyssinian_127.jpg 0
Abyssinian_128.jpg 0
Abyssinian_129.jpg 0
Abyssinian_12.jpg 0
Abyssinian_130.jpg 0
Abyssinian_131.jpg 0
Abyssinian_132.jpg 0
Abyssinian_133.jpg 0
Abyssinian_134.jpg 0
Abyssinian_135.jpg 0
Abyssinian_136.jpg 0
Abyssinian_137.jpg 0
Abyssinian_138.jpg 0
Abyssinian_139.jpg 0
Abyssinian_13.jpg 0
Abyssinian_140.jpg 0
Abyssinian_141.jpg 0
Abyssinian_142.jpg 0
Abyssinian_143.jpg 0
Abyssinian_144.jpg 0
Abyssinian_145.jpg 0
american_bulldog_

In [173]:
import os
class PetsDataset(data.Dataset):
    def __init__(self, df, path_to_images,  transform=None):
        self.path = path_to_images
        self.transform = transform

        data = df
        
        self.length = len(data)

        self.files = []

        
        for file, target in zip(data.image, data.class_id):
            tmp_path = os.path.join(self.path, file)
            
            self.files.append((tmp_path, target))

            


    def __getitem__(self, item):
        path_file, target = self.files[item]
        

        img = Image.open(path_file)

        if self.transform:
            img = self.transform(img).float() 
        
        target = torch.tensor(target, dtype= torch.long)
        return img, target

    def __len__(self):
        return self.length

In [174]:
from torchvision import transforms as tfs

transform = tfs.Compose([
    tfs.Resize((224, 224)),  # или любой другой размер
    tfs.Lambda(lambda x: x.convert('RGB')),  # Конвертируем в RGB
    tfs.RandomHorizontalFlip(),
    tfs.ToTensor(),  # PILToTensor

    tfs.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
])

d_train = PetsDataset(train, "data\\images", transform=transform)
d_test = PetsDataset(test, "data\\images", transform=transform)

train_data_loader = data.DataLoader(d_train, batch_size=32, shuffle=True)
test_data_loader = data.DataLoader(d_test, batch_size=32, shuffle=True)

print(len(d_train))

5509


In [175]:
for X, y in train_data_loader:
        
    print(X.device.type, y.device.type)
    print(y)
    break

cpu cpu
tensor([21,  2, 31, 16,  0, 27, 10, 25, 25,  6,  5, 32,  9,  9,  3, 18, 19,  2,
         5, 23, 24, 36, 24, 17, 36, 25, 32, 35,  4, 35, 21, 14])


# Дообучение

In [176]:
import torchvision.models as models


In [177]:
alexnet = torchvision.models.alexnet(weights=True).to(device)
vgg = torchvision.models.vgg16(weights=True).to(device)
resnet = torchvision.models.resnet50(weights=True).to(device)


c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other tha

In [178]:
vgg.classifier[-1] = nn.Linear(4096, 37)  
nn.init.normal_(vgg.classifier[6].weight, 0, 0.01)
vgg.to(device)


alexnet.classifier[-1] = nn.Linear(4096, 37)  
nn.init.normal_(alexnet.classifier[6].weight, 0, 0.01)
alexnet.to(device)

resnet.fc = nn.Linear(2048, 37)  
resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [179]:
# for batch, (X, y) in enumerate(train_data_loader):
#     # Compute prediction and loss
#     pred = alexnet(X)
#     print(pred.size())
#     print(y.size())
#     break


In [180]:
alexnet.classifier



Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=37, bias=True)
)

In [181]:
alexnet._get_name()

'AlexNet'

In [182]:
vgg._get_name()

'VGG'

In [183]:
resnet._get_name()

'ResNet'

In [184]:
import csv
import os
import torch
import torch.nn as nn

epochs = 50
loss_function = nn.CrossEntropyLoss()

models = [resnet, vgg, alexnet]
csv_file = "models/metrics/models_metrics.csv"

# Создаем директорию если не существует
os.makedirs("models/metrics", exist_ok=True)

fieldnames = ['model_name', 'accuracy', 'loss', 'precision_weighted', 'recall_weighted', 'f1_weighted']

# Проверяем существует ли файл, чтобы определить нужно ли писать заголовок
file_exists = os.path.isfile(csv_file)

PATIENCE = 5

with open(csv_file, 'a', newline='', encoding='utf-8') as csv_f:
    writer = csv.DictWriter(csv_f, fieldnames=fieldnames)
    
    if not file_exists:
        writer.writeheader()
    
    for model in models:
        model_name = model._get_name()
        
        model_exist = os.path.isfile(f"models/final_{model_name}.pth")
        if model_exist:
            continue

        print(f"Началось обучение {model_name}")

        if model_name == 'ResNet':
            optimizer = torch.optim.Adam(model.parameters())
        else:
            optimizer = torch.optim.Adam(model.classifier.parameters())
        
        # Переменные для отслеживания лучшей модели
        best_accuracy = 0.0
        best_metrics = None
        best_epoch = 0
        
        for epoch in range(epochs):
            train_loop(train_data_loader, model, loss_function, optimizer)
            
            # Получаем метрики после каждой эпохи
            test_metrics = test_loop(test_data_loader, model, loss_fn=loss_function)
            
            current_accuracy = test_metrics['accuracy']
            
            # Сохраняем лучшую модель (по accuracy)
            if current_accuracy > best_accuracy:
                best_accuracy = current_accuracy
                best_metrics = test_metrics
                best_epoch = epoch + 1
                
                # Сохраняем веса лучшей модели
                torch.save(model.state_dict(), f'models/best_{model_name}_epoch{best_epoch}.pth')
                print(f"Новая лучшая модель {model_name} на эпохе {best_epoch} с accuracy: {best_accuracy:.4f}")
                patience = 0
                
            else:
                patience += 1
                if patience >= PATIENCE:
                    print("Early stopping")
                    break   

            
        
        # После завершения всех эпох сохраняем финальную модель
        torch.save(model.state_dict(), f'models/final_{model_name}.pth')
        
        # Записываем метрики ЛУЧШЕЙ модели в CSV
        row_data = {'model_name': f"{model_name}_best_epoch{best_epoch}"}
        row_data.update(best_metrics)

        writer.writerow(row_data)
        
        print(f"Лучшая модель {model_name} сохранена с эпохи {best_epoch}, accuracy: {best_accuracy:.4f}")


Началось обучение AlexNet
loss: 4.297191  [   32/ 5509]
loss: 2.020835  [ 1632/ 5509]
loss: 1.300406  [ 3232/ 5509]
loss: 1.643317  [ 4832/ 5509]
Test Error: 
 Accuracy: 62.3%, Avg loss: 1.179865 

weighted averages
Precision: 0.7067
Recall:    0.6228
F1:        0.6095
Новая лучшая модель AlexNet на эпохе 1 с accuracy: 0.6228
loss: 1.514973  [   32/ 5509]
loss: 1.285630  [ 1632/ 5509]
loss: 1.107569  [ 3232/ 5509]
loss: 1.720088  [ 4832/ 5509]
Test Error: 
 Accuracy: 66.5%, Avg loss: 1.062549 

weighted averages
Precision: 0.7221
Recall:    0.6647
F1:        0.6556
Новая лучшая модель AlexNet на эпохе 2 с accuracy: 0.6647
loss: 1.109851  [   32/ 5509]
loss: 0.561104  [ 1632/ 5509]
loss: 1.613542  [ 3232/ 5509]
loss: 1.678803  [ 4832/ 5509]
Test Error: 
 Accuracy: 67.1%, Avg loss: 1.100993 

weighted averages
Precision: 0.7321
Recall:    0.6707
F1:        0.6668
Новая лучшая модель AlexNet на эпохе 3 с accuracy: 0.6707
loss: 2.040432  [   32/ 5509]
loss: 0.587439  [ 1632/ 5509]
loss: 1.

KeyboardInterrupt: 

In [158]:
alexnet_not_pretrained = torchvision.models.alexnet(weights=None).to(device)
vgg_not_pretrained = torchvision.models.vgg16(weights=None).to(device)
resnet_not_pretrained = torchvision.models.resnet50(weights=None).to(device)


vgg_not_pretrained.classifier[6] = nn.Linear(4096, 37)  
nn.init.normal_(vgg_not_pretrained.classifier[6].weight, 0, 0.01)
vgg_not_pretrained.to(device)


alexnet_not_pretrained.classifier[6] = nn.Linear(4096, 37)  
nn.init.normal_(alexnet_not_pretrained.classifier[6].weight, 0, 0.01)
alexnet_not_pretrained.to(device)

resnet_not_pretrained.fc = nn.Linear(2048, 37)  
resnet_not_pretrained.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Я очень долго пытался обучать с начала, но я честно не понимаю почему не меняется лосс. Пробовал очень много всего.

In [159]:
def init_xavier(model):
    """Инициализация всех линейных и сверточных слоев с помощью Xavier"""
    for module in model.modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)
                
init_xavier(alexnet)

In [164]:
epochs = 100
loss_function = nn.CrossEntropyLoss()

models = [alexnet_not_pretrained]   # vgg_not_pretrained, resnet_not_pretrained
csv_file = "models/metrics/models_metrics.csv"

# Создаем директорию если не существует
os.makedirs("models/metrics", exist_ok=True)
fieldnames = ['model_name', 'accuracy', 'loss', 'precision_weighted', 'recall_weighted', 'f1_weighted']

# Проверяем существует ли файл, чтобы определить нужно ли писать заголовок
file_exists = os.path.isfile(csv_file)
PATIENCE = 5

with open(csv_file, 'a', newline='', encoding='utf-8') as csv_f:
    
    writer = csv.DictWriter(csv_f, fieldnames=fieldnames)
    
    if not file_exists:
        writer.writeheader()
    
    for model in models:
        model_name = model._get_name() + '_not_pretrained'
        
        model_exist = os.path.isfile(f"models/final_{model_name}.pth")
        if model_exist:
            continue

        print(f"Началось обучение {model_name}")

        optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)
        
        
        # Переменные для отслеживания лучшей модели
        best_accuracy = 0.0
        best_metrics = None
        best_epoch = 0
        
        for epoch in range(epochs):
            train_loop(train_data_loader, model, loss_function, optimizer)
            
            # Получаем метрики после каждой эпохи
            test_metrics = test_loop(test_data_loader, model, loss_fn=loss_function)
            
            current_accuracy = test_metrics['accuracy']
            
            # Сохраняем лучшую модель (по accuracy)
            if current_accuracy > best_accuracy:
                best_accuracy = current_accuracy
                best_metrics = test_metrics
                best_epoch = epoch + 1
                
                # Сохраняем веса лучшей модели
                torch.save(model.state_dict(), f'models/best_{model_name}_epoch{best_epoch}.pth')
                print(f"Новая лучшая модель {model_name} на эпохе {best_epoch} с accuracy: {best_accuracy:.4f}")
                patience = 0
                
            # else:
            #     patience += 1
            #     if patience >= PATIENCE:
            #         print("Early stopping")
            #         break   

        
        # После завершения всех эпох сохраняем финальную модель
        torch.save(model.state_dict(), f'models/final_{model_name}.pth')
        
        # Записываем метрики ЛУЧШЕЙ модели в CSV
        row_data = {'model_name': f"{model_name}_best_epoch{best_epoch}"}
        row_data.update(best_metrics)

        writer.writerow(row_data)
        
        print(f"Лучшая модель {model_name} сохранена с эпохи {best_epoch}, accuracy: {best_accuracy:.4f}")


Началось обучение AlexNet_not_pretrained
loss: 3.609633  [   32/ 5509]
loss: 3.610131  [ 1632/ 5509]
loss: 3.607043  [ 3232/ 5509]
loss: 3.609190  [ 4832/ 5509]
Test Error: 
 Accuracy: 2.7%, Avg loss: 3.610767 

weighted averages
Precision: 0.0007
Recall:    0.0272
F1:        0.0014
Новая лучшая модель AlexNet_not_pretrained на эпохе 1 с accuracy: 0.0272
loss: 3.611688  [   32/ 5509]
loss: 3.613858  [ 1632/ 5509]
loss: 3.610164  [ 3232/ 5509]
loss: 3.611374  [ 4832/ 5509]
Test Error: 
 Accuracy: 2.7%, Avg loss: 3.610767 

weighted averages
Precision: 0.0007
Recall:    0.0272
F1:        0.0014
loss: 3.608871  [   32/ 5509]
loss: 3.611790  [ 1632/ 5509]
loss: 3.620384  [ 3232/ 5509]
loss: 3.612836  [ 4832/ 5509]
Test Error: 
 Accuracy: 2.7%, Avg loss: 3.610804 

weighted averages
Precision: 0.0007
Recall:    0.0272
F1:        0.0014
loss: 3.610031  [   32/ 5509]
loss: 3.610756  [ 1632/ 5509]
loss: 3.614758  [ 3232/ 5509]
loss: 3.611068  [ 4832/ 5509]
Test Error: 
 Accuracy: 2.7%, Avg los

KeyboardInterrupt: 

In [ ]:
pd.read_csv('models/metrics/models_metrics.csv')

,model_name,accuracy,loss,precision_weighted,recall_weighted,f1_weighted
0,ResNet_best_epoch3,0.483696,1.836274,0.610984,0.483696,0.475211
1,AlexNet_best_epoch3,0.561821,1.421071,0.648144,0.561821,0.551330
2,VGG_best_epoch3,0.654212,1.124183,0.722936,0.654212,0.638382
3,AlexNet_not_pretrained_best_epoch1,0.027174,3.610932,0.000738,0.027174,0.001438
4,AlexNet_best_epoch1,0.673381,1.085364,0.734371,0.673381,0.670024
5,AlexNet_not_pretrained_best_epoch1,0.027218,3.610910,0.000741,0.027218,0.001442
6,AlexNet_not_pretrained_best_epoch2,0.027218,3.610783,0.000741,0.027218,0.001442


# Был создан кастомный датасет с классификацией животных. Были дообучены 3 модели resnet alexnet vgg. метрики для разных моделей. Обучались модели не очень много эпох, но результат виден, лучше всего показала себя vgg. Она как раз таки обучалась дольше всех.